In [19]:
import os
import csv
import shutil
from time import time
from utils.process_label_to_txt import convert_txt
from demucs_utils.seperate_vocal import separate
from mfa.align import create_parser, run_align_corpus
from mfa.src.postprocessing import post_process_helper

In [20]:
PUBLIC_TEST = "data/public_test"
SONG_RAW_DIR = "data/public_test/songs"
RAW_LYRIC_JSON = "data/public_test/json_labels"

SEPARATED_DATA_DIR = "data/output"
PUBLIC_TEST_OUTPUT_RAW = "data/output/public_test_raw"
OUTPUT_DIR = "data/output/public_test_json"

SUBMISSION_DIR = "./result"
OUTPUT_FILE = "./result/submission.zip"
OUTPUT_TIME_SUBMISSION = "./result/time_submission.csv"
OUTPUT_JUPYTER_FILE = "./result/jupyter_submission"

In [21]:
test_cases = os.listdir(SONG_RAW_DIR)
len(test_cases)

['37303234365f3231.wav', '37303234365f3431.wav', '37303234365f3732.wav']

In [22]:
all_predicted_time = []
parser = create_parser()
args, unknown = parser.parse_known_args(["align"])
args.output_directory = PUBLIC_TEST_OUTPUT_RAW

for file_name in test_cases:
    t1 = time()
    separate(os.path.join(SONG_RAW_DIR, file_name), SEPARATED_DATA_DIR)
    current_dir_separate = os.path.join(SEPARATED_DATA_DIR, "mdx_extra_q", file_name[:-4])
    separate_optimized_dir = os.path.join(current_dir_separate, file_name[:-4])
    os.makedirs(separate_optimized_dir)
    os.popen("ffmpeg -i {input} -ar 16000 -ac 1 -y {output}".format(input=os.path.join(current_dir_separate, "vocals.wav"), output=os.path.join(separate_optimized_dir, file_name)))
    convert_txt(file_name.replace("wav", "json"), RAW_LYRIC_JSON, separate_optimized_dir)
    args.corpus_directory = separate_optimized_dir
    run_align_corpus(args, unknown)
    post_process_helper.post_process_json(
        file_name=file_name[:-4], 
        raw_output=PUBLIC_TEST_OUTPUT_RAW, 
        raw_lyric=RAW_LYRIC_JSON,
        output_dir=OUTPUT_DIR,
    )
    t2 = time()
    predicted_time = int(t2*1000 - t1*1000)
    all_predicted_time.append((file_name, predicted_time))

if not os.path.exists(SUBMISSION_DIR):
    os.makedirs(SUBMISSION_DIR)
    
# Save time submission
with open(OUTPUT_TIME_SUBMISSION, 'w') as f:
    write = csv.writer(f)
    fields = ["fname", "time (millisecond)"] 
    write.writerow(fields)
    write.writerows(all_predicted_time)

# Save jupyter submission
shutil.make_archive(OUTPUT_JUPYTER_FILE, 'zip', OUTPUT_DIR)

Separate audio: data/public_test/songs/37303234365f3231.wav
With command:  python -m demucs.separate -o data/output -n mdx_extra_q -j 2 --float32 --two-stems=vocals
Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /mnt/c/Users/Modern 14/projects/CTA-Zero9-ZAIC2022-Lyric-Alignment/data/output/mdx_extra_q
Separating track data/public_test/songs/37303234365f3231.wav


100%|██████████████████████████████████████████████████████████████████████████| 33.0/33.0 [00:18<00:00,  1.81seconds/s]
100%|██████████████████████████████████████████████████████████████████████████| 33.0/33.0 [00:35<00:00,  1.08s/seconds]
100%|██████████████████████████████████████████████████████████████████████████| 33.0/33.0 [00:22<00:00,  1.45seconds/s]
100%|██████████████████████████████████████████████████████████████████████████| 33.0/33.0 [00:17<00:00,  1.85seconds/s]
ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-18)
  configuration: --prefix=/home/vnk/miniconda3/envs/lyric-test --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-nm --

INFO - Setting up corpus information...
INFO - Loading corpus from source files...


100%|██████████| 1/1 [00:01<00:00,  1.08s/it]

INFO - Found 1 speaker across 1 file, average number of utterances per speaker: 1.0
INFO - Initializing multiprocessing jobs...
WARNING - Number of jobs was specified as 3, but due to only having 1 speakers, MFA will only use 1 jobs.
INFO - Creating corpus split for feature generation...
INFO - Generating base features (mfcc)...
INFO - Generating MFCCs...


...


100%|██████████| 1/1 [00:02<00:00,  2.14s/it]


INFO - Exporting TextGrids to data/output/public_test_raw...
INFO - Collecting phone and word alignments from alignment lattices...


100%|██████████| 1/1 [00:00<00:00, 15.64it/s]

INFO - Finished exporting TextGrids to data/output/public_test_raw!
INFO - Done! Everything took 32.1484808921814 seconds


'/mnt/c/Users/Modern 14/projects/CTA-Zero9-ZAIC2022-Lyric-Alignment/result/jupyter_submission.zip.zip'